In [ ]:
import numpy as np

arr = np.arange(32).reshape((8, 4))

In [ ]:
arr

In [ ]:
print( arr[ [ 1, 5, 7, 2 ] ] [ :, [ 0, 3, 1, 2 ] ] )

In [ ]:
arr[[:],[0,1]]

In [ ]:
    products = {"Black": ["dispensing cup","dispensing coffee", "dispensing water",],
                "WhiteSweet": ["dispensing cup", "dispensing coffee", "dispensing sugar", "dispensing water",],
                "Sweet": ["Dispensing cup", "Dispensing coffee", "Dispensing sugar", "dispensing water"]}

    print(products)

    for product, details in products.items():
        print(f'\nPRODUCT: {product}')
        for detail in details:
            print(f'\t{detail}')

In [ ]:
a=5
b = [['.' for _ in range(a)] for _ in range(a)]
b2 = []
for _ in range(a):
    c = []
    for _ in range(a):
        c.append('.')
    b2.append(c)
b2 == b

In [ ]:
d = ['.' for _ in range(a)] * a
d==b

In [ ]:
d

In [ ]:
    import re

    regex = r"^\d+([a-zA-Z])+\d+$"
    test_str = "g1a2"

    ok = re.search(regex, test_str)

In [ ]:
    class Pizza:
        def __init__(self):
            self.size = 0
            self.toppings = ''
            self.flavour = ''

        def __repr__(self):
            return f'Size: {self.size}, toppings: {self.toppings}, flavour: {self.flavour}'

        def __str__(self):
            return f'{self.size},"{self.toppings}","{self.flavour}"'

    pizza_orders = []
    pizza_orders.append(Pizza())  # class names usuall start with capital letter
    pizza_orders[-1].size = 5
    pizza_orders[-1].toppings = 'stuff'
    pizza_orders[-1].flavour = 'tasty'

    for pizza in pizza_orders:
        print(my_pizza)

    with open('pizza_orders.csv', 'w') as p:
        for pizza in pizza_orders:
            p.write(f'{pizza}\n')

In [ ]:
cat pizza_orders.csv

In [ ]:
    import os


    def intro():
        print("""
    Welcome to the Cashless system Prototype :

    In this prototype you have different options to choose from:

    \tOption 1, enter 1 to Create a user
    \tOption 2, enter 2 to Add Balance
    \tOption 3, enter 3 to Widthdraw Balance
    \tOption 4, enter 4 to View your balance
    \tOption 5, enter 5 to Viewing your data that is stored
    \tOption 6, enter 6 to Delete your data that is stored
    \tOption 7, enter 7 to Append data to the text file that stores data
    """)

    def createuser():
        print()
        ID = input("Enter your College ID Number: ")
        FirstName = input("Enter your first name: ")
        SecondName = input("Enter your second name: ")
        Gender = input("Enter your gender M for male and F for female: ").upper()
        Day = input("Enter the day which you were born in: ")
        Month = input("Enter the number of the month you were born in: ")
        Year = input("Enter the year you were born in: ")
        with open("StudentData.txt","w") as file:
            file.write(f'{ID}\n{FirstName} {SecondName}\n{Gender}\n{Day}/{Month}/{Year}')

    def add():
        if not os.path.isfile("Balance.txt"):
            with open("Balance.txt","w") as file:
                file.write("0")

        with open("Balance.txt") as bal:
            Balance1 = float(bal.read().strip())
        print(f"Your current balance is £{Balance1:.2f}")
        Amount = float(input("Enter how much you want to add to your account: "))
        NewBalance = Balance1 + Amount
        print(f"Your new balance is £{NewBalance:.2f}")
        with open("Balance.txt","w") as bal:
            bal.write(f"{NewBalance}")

    #THIS IS THE PART WHERE THE PROGRAM BEGINS
    intro()
    while True:
        Option = input("Enter what option you want:")
        if Option == "1":
            createuser()
        elif Option == "2":
            add()
        else:
            print('Sorry, option not available')
        Choice = input("Enter yes or no if you want the program to carry on or end it: ").lower()
        if Choice in ('no', 'nope', 'quit', 'bye', 'exit'):
            break

In [ ]:
import string

def ispangram(str1, alphabet=string.ascii_lowercase):  
    alphaset = set(alphabet)  
    return alphaset <= set(str1.lower())

In [ ]:
def averages(*args):
    avs = []
    for list_ in args:
        avs.append(sum(list_) / len(list_))
    return sum(avs)/len(avs)

print(averages([1,2,3], [4,3,3], [1,1,1]))

In [6]:
from tkinter import *
import csv

class user_information():
    def __init__(self, username_db, password_db):
        self.username_db = username_db
        self.password_db = password_db

    def readCSVFile():
        users = []
        with open('userinfo.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for user_name, user_password in csv_reader:
                currentUser = user_information(user_name, user_password)
                users.append(currentUser)
        return users

    def register_validation(username_info, users):
        if username_info:
            for user in users:
                if user.username_db == username_info:
                    break
            else:  # for loop completed with no matches
                return True
        return False
    
def register_user():
    userList = readCSVFile()   
    username_info = username_entry.get()
    password_info = password_entry.get()
    passed = register_validation(username_info, userList)
    if passed:
        with open('userinfo.csv', 'a') as csvfile:
            spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
            spamwriter.writerow([username_info] +[","]+ [password_info])
        
        username_entry.delete(0, END)
        password_entry.delete(0, END)
        Label(register_screen, text = "Registration successful!", bg="white", fg="lightgreen").pack()
    else:
        Label(register_screen, text = "Username must be valid, please try again", bg="white", fg="red").pack()

def register():
    
    global register_screen
    
    register_screen = Toplevel(home)
    register_screen.title("Register")
    register_screen.geometry("300x250")
    register_screen.configure(bg="white")

    global username
    global password
    global username_entry
    global password_entry
    username = StringVar()
    password = StringVar()

    Label(register_screen, text="Please enter details below", fg="white", bg="lightblue", width="300", height="2",font=("Verdana", 13)).pack()
    Label(register_screen, text="", bg="white").pack()

    username_label = Label(register_screen, text="Username * ", bg=("white"))
    username_label.pack()

    username_entry = Entry(register_screen, textvariable=username)
    username_entry.pack()

    password_label = Label(register_screen, text="Password * ", bg=("white"))
    password_label.pack()

    password_entry = Entry(register_screen, textvariable=password, show='*')
    password_entry.pack()

    Label(register_screen, text="", bg=("white")).pack()

    Button(register_screen, text = "Register", width=10, height=1, command=register_user).pack()

def login():
    print('login')
    
def home_screen():
    global home
    home = Tk()
    home.geometry('300x250')
    home.title("Welcome!")
    home.configure(bg="white")
                   
    Label(text="Login or Register?", fg="white", bg="lightblue", width="300", height="2",font=("Verdana", 13)).pack()
    Label(text="", bg="white").pack()

    Button(home, text="Login", height="2", width="30", relief="groove", command=login).pack()
    Label(text="", bg="white").pack()
    
    Button(text="Register", height="2", width="30", relief="groove", command=register).pack()

#Main Program

print('RUNNING')
home_screen()
home.mainloop()
print('FINI')

RUNNING
reading...
fred pass1
bob pass2
kev pass3
fred is already used
reading...
fred pass1
bob pass2
kev pass3
freddy is new
reading...
fred pass1
bob pass2
kev pass3
freddy   pass4
bob is already used
reading...
fred pass1
bob pass2
kev pass3
freddy   pass4
bobby is new
FINI


In [5]:
data="""fred,pass1
bob,pass2
kev,pass3
"""
with open('userinfo.csv','w') as f:
    f.write(data)

In [4]:
cat userinfo.csv

fred,pass1
bob,pass2
kev,pass3fred1 , pass


In [101]:
def daysToReview(message, year=2020):

    def define_months(year):
        """return dict of months: number of days for <year>"""
        months = {'Jan': 31, 'Feb': 28, 'Mar': 31,
                  'Apr': 30, 'May': 31, 'Jun': 30,
                  'Jul': 31, 'Aug': 31, 'Sep': 30,
                  'Oct': 31, 'Nov': 30, 'Dec': 31}
        if year % 4 == 0:
            if year % 100 != 0 or year % 400 == 0:
                months['Feb'] = 29
        return months

    def extract_dates_from_msg(msg):
        """extract & return two validated <month> and <day_of_month> pairs
        from <msg>
        """

        def is_valid_day(month, day):
            return 1 <= day <= months[months_index[month]]

        detected_months = [month for month in split_msg[2:] if month in months]
        detected_days = [int(day) for day in split_msg[2:] if day.isdigit()]

        if not (len(detected_months) == 2 and len(detected_days) == 2):
            raise ValueError('incomplete start and finish months and days info')

        start, start_day = months_index.index(detected_months[0]), detected_days[0]
        fini, fini_day  = months_index.index(detected_months[1]), detected_days[1]

        if not is_valid_day(start, start_day):
            raise ValueError('start date is not valid')

        if not is_valid_day(fini, fini_day):
            raise ValueError('finish date is not valid')

        # put months / days correct way around for difference calc
        if start == fini and start_day > fini_day:
            start_day, fini_day = fini_day, start_day
        elif start > fini:
            start, fini = fini, start
            start_day, fini_day = fini_day, start_day

        return start, start_day, fini, fini_day


    months = define_months(year)
    months_index = tuple(months)
    split_msg = message.title().split()
    firstname, lastname = split_msg[1], split_msg[2]  # what is at [0]?

    try:
        start, start_day, fini, fini_day = extract_dates_from_msg(split_msg[2:])
    except ValueError as e:
        raise ValueError(e)

    if start == fini:
        diff = fini_day - start_day
    else:
        diff = months[months_index[start]] - start_day + 1
        for mnt in range(start + 1, fini):
            diff += months[months_index[mnt]]
        diff += fini_day

    return diff

tests = ('something Fred Blogs Jan 10 to febr 12',
         'something Bert Jones feb 29 to feb 12',
         'something Sara Wills MAR 32 to Jun 12',
         'something Bryn Harts Nov 21 to Aug 23',
        )

for test in tests:
    print(f'Trying {test}, difference = ', end='')
    try:
        print(daysToReview(test))
    except ValueError as e:
        print(e)

Trying something Fred Blogs Jan 10 to febr 12, difference = incomplete start and finish months and days info
Trying something Bert Jones feb 29 to feb 12, difference = 17
Trying something Sara Wills MAR 32 to Jun 12, difference = start date is not valid
Trying something Bryn Harts Nov 21 to Aug 23, difference = 91
